# String algorithms
Choices:
- trie *
- hashing function (hash table)
- bloom filters
- cuckoo hash table

( * researching)

# The trie algorithm

## https://www.cs.bu.edu/teaching/c/tree/trie/

### What is trie?

  - The term trie comes from retrieval.
  - Works with key, value pairs.
  - Keys are strings (in this example).
  - Sharing prefixes that are common among keys.
  - Using a tree.
  - Every character in the key (string) is a node.
  - Keys always end with a nul character (\0) node.
  - Data is saved in nul node.

Example data (names + ages):
  ```
  amy   56
  ann   15
  emma  30
  rob   27
  roger 52
```
  
Example trie for `amy`:
  ```
    .  <- root
    |
    a
    |
    m
    |
    y
    |
  \0 56
```

Example trie for `amy` and `ann`:
  ```
      .
       |
       a
     /   \
    m     n
    |     |
    y     n
    |     |
  \0 56 \0 15
```

End trie for example:

  ```
                .
        /       |      \
       a        e       r
     /   \      |       |
    m     n     m       o
    |     |     |     /   \
    y     n     m    b     g
    |     |     |    |     |
  \0 56 \0 15   a  \0 27   e
                |          |
              \0 30        r
                           |
                         \0 52
```

#### Questions

##### What would the trie look like if we now added __anne__ with age __67__? How about __ro__ with age __23__? 
  ```
                   .
        /          |         \
       a           e          r
     /    \        |          |
    m      n       m          o
    |      |       |     /    |   \
    y      n       m    b     g \0 23
    |    /    \    |    |     |
  \0 56 \0 15  e   a  \0 27   e
              /    |          |
           \0 67 \0 30        r
                              |
                            \0 52
```

##### Would the trie look different if we added the names in a different order, say: __rob__, __ann__, __emma__, __roger__, __amy__? 
No the order of the names does not change this tree.

##### Is this a binary tree, tertiary tree or what? In other words, each node has __at most__ how many children? 
This dependece on the alphabet used, in the english alphabet the maximum amount of children would be __26__.

### Trie operations:

- Add: Add data to the trie.
- IsMember: check if key is in trie.
- Remove: remove key from the trie.

In this example IsMember only return true or false, we could also implement a function to get the value from this key.

#### Questions:

#####  Since our trie holds data with string keys, which of the operations need __a key and value__, and which just need keys? 
Only adding a value needs a value, the other two operations only need the key.

## https://en.wikipedia.org/wiki/Trie

### Advantages of a trie:
- Speed worst case `O(m)` (`m` is length of string) is better than an imperfect hash table. 
- Alphabetic ordering.


In [67]:
# A trie implementation in python
import itertools

class TrieIndexError(IndexError):
    
    def __init__(self, key, name):
        super().__init__(
            '"{}" does not exist in trie'.format(key)
        )
        self.name = name
        self.key = key

class Node(object):
    
    def __init__(self, name, value=None, parent=None):
        self.parent = parent
        self.name = name
        self.children = dict()
        if not self.parent:
            self.key = self.name
        else:
            self.key = self.parent.key + self.name
        self.position = len(self.key)
  
    def _get_next_node(self, key):
        next_name = key[self.position:self.position + 1]
        if next_name in self.children:
            node = self.children[next_name]
        else:
            raise TrieIndexError(key, next_name)
        return node

    def _create_child(self, name, value=None):
        
        if name:
            node = Node(name, parent=self)
        else:
            node = EndNode(name, value, parent=self)
        self.children[name] = node
        return node
        
    def __contains__(self, key):
        try:
            node = self._get_next_node(key)
        except TrieIndexError:
            return False
        return node.__contains__(key)
    
    def __getitem__(self, key):
        node = self._get_next_node(key)
        return node.__getitem__(key)
        
    def __setitem__(self, key, value):
        try:
            node = self._get_next_node(key)
        except TrieIndexError as e:
            node = self._create_child(e.name, value=value)
        node.__setitem__(key, value)
        
    def __delitem__(self, key):
        """Go down all the nodes then up again deleting."""
        node = self._get_next_node(key)
        node.__delitem__(key)

    def _del_node(self, name):
        """Only delete nodes without children."""
        node = self.children[name]
        if not node.children:
            del self.children[name]
            
    def get_subnode(self, key):
        if key == self.key:
            return self
        node = self._get_next_node(key)
        return node.get_subnode(key)
    
    
class EndNode(Node):
    
    def __init__(self, name, value, parent):
        super().__init__(name, value, parent)
        self.value = value
        
    def __setitem__(self, key, value):
        self.value = value
    
    def __getitem__(self, key):
        return self.value
    
    def __contains__(self, key):
        return True
    
    def __delitem__(self, key):
        """Start going up for node deletion."""
        self.parent._del_node(self.name)
        

# Create trie
trie = Node('')

# Add values
trie['bla'] = 4
trie['hello'] = 5
trie['hell'] = 20
trie['blabla'] = 8
trie['bla'] = 10

# Test getting values
print(trie['bla'])
print(trie['blabla'])
print(trie['hello'])
print(trie['hell'])

# Test contains
print('hello' in trie)
print('hell' in trie)
print('bla' in trie)
print('blabla' in trie)


# Test that getting keys that do not exist fail
def test_fail_get(trie, key):
    
    try:
        trie[key]
    except TrieIndexError:
        pass
    else:
        raise Exception('Should not find key "{}"!'.format(key))

test_fail_get(trie, 'bl')
test_fail_get(trie, 'hel')
test_fail_get(trie, '')
test_fail_get(trie, 'blab')
test_fail_get(trie, 'helloo')
# Test deletes
del trie['bla']
print('bla' in trie)
print('blabla' in trie)

del trie['hello']
print('hello' in trie)
print('hell' in trie)

# Test get a submodule and find keys in it
sb = trie.get_subnode('he')
print(sb['hell'])
print(trie)

10
8
5
20
True
True
True
True
False
True
False
True
20


In [64]:
position = len('')
'abcde'[position:position+1]
list(itertools.chain.from_iterable({'b': {'c': 'a'}, 'd': {}}.items()))

['d', {}, 'b', {'c': 'a'}]

# Bloom filter

These are not key value pairs but just values!
Creates a bit array.
Can only add values.
And check if a value is contained in the array:
- with a probability: it is possible to have false positives.

- A pre set list of hash functions is used to determine if a value contained, for example `[h1, h2, h3]` 
- On `add(a)` calculate the hash of `a` for all `h1`, `h2` and `h3`. Every hash returns a location on the array, this location is set to `True`.
- On `contains(a)` calculate all hashes of `a`. If any location gotten from the hashes returns `false` we are sure that `a` is not in the bloom filter, else it is possible it has been added.
(Comparable with `sets` in Python.)

# Hash table
key value pairs
Save keys in an array.
Uses __one__ hash on the key to determine where on the array to save the value.
It can happen that different keys have the same hash: this is a collition.